# BenzDB tools

## Source code

### Module imports

In [1]:
import ipywidgets as w
import requests
import json
import base64 
import io
from PIL import Image
import IPython as ip
import jupyter_jsmol as Jsmol
import pandas as pd
import matplotlib.pyplot as plt

### Definition of criterion classes

#### Definition of the main criterion class

In [2]:
class Criterion:
    """ This class allows for representing criterions """
    
    def __init__ (self, key: str, description: str):
        """ initializes the criterion """
        self.__key = key
        self.__description  = description     
    
    
    def display (self):
        """ displays the widget corresponding to the criterion """
        pass
    
    
    def get_criterion (self) -> str:
        """ returns the JSON string corresponding to the criterion, an empty string if the criterion is not set """
        pass
    
    
    def get_description (self) -> str:
        """ returns the description """
        return self.__description
    
    def get_key (self) -> str:
        """ returns the key related to the criterion """
        return self.__key

#### Definition of criterion child classes

In [3]:
class Int_Criterion (Criterion):
    """ This class allows for representing criterions based on int value """
 
    def __init__ (self, key: str, description: str, min_value: int, max_value: int):
        """ initializes the criterion """
        super().__init__(key, description)
        
        self.__element =  w.BoundedIntText(value=min_value, min=min_value, max=max_value, layout={"width": "auto"})
        self.__condition = w.Select(description=self.get_description(), options=["not set","=","<>","<=","<",">",">="], value='not set', rows=1, layout={"width": "auto"})
                
        box = w.HBox([self.__condition, self.__element])
        display(box)
        
    
    def get_criterion (self) -> str:
        """ returns the JSON string corresponding to the criterion, an empty string if the criterion is not set """
        if self.__condition.value == "not set":
            return ""
        else:
            return '"' + self.get_key() + '": "' + self.__condition.value + " " + str(self.__element.value) + '"'

In [4]:
class Float_Criterion (Criterion):
    """ This class allows for representing criterions based on float value """
 
    def __init__ (self, key: str, description: str, min_value: float, max_value: float):
        """ initializes the criterion """
        super().__init__(key, description)
        
        self.__element =  w.BoundedFloatText(value=min_value, min=min_value, max=max_value, layout={"width": "auto"})
        self.__condition = w.Select(description=self.get_description(), options=["not set","=","<>","<=","<",">",">="], value='not set', rows=1, layout={"width": "auto"})
                
        box = w.HBox([self.__condition, self.__element])
        display(box)
        
    
    def get_criterion (self) -> str:
        """ returns the JSON string corresponding to the criterion, an empty string if the criterion is not set """
        if self.__condition.value == "not set":
            return ""
        else:
            return '"' + self.get_key() + '": "' + self.__condition.value + " " + str(self.__element.value) + '"'

In [5]:
class String_Criterion (Criterion):
    """ This class allows for representing criterions based on string value """
 
    def __init__ (self, key: str, description: str):
        """ initializes the criterion """
        super().__init__(key, description)
        
        self.__element =  w.Text(layout={"width": "auto"})
        self.__condition = w.Select(description=self.get_description(), options=["not set","=","<>"], value='not set', rows=1, layout={"width": "auto"})
                
        box = w.HBox([self.__condition, self.__element])
        display(box)
        
    
    def get_criterion (self) -> str:
        """ returns the JSON string corresponding to the criterion, an empty string if the criterion is not set """
        if self.__condition.value == "not set":
            return ""
        else:
            return '"' + self.get_key() + '": "' + self.__condition.value + " " + str(self.__element.value) + '"'

In [6]:
class Query_Criterion (Criterion):
    """ This class allows for representing the desired query """
 
    def __init__ (self, key: str, description: str):
        """ initializes the criterion """
        super().__init__(key, description)
        
        self.__element =  w.Select(description=self.get_description(), options=["benzenoids","ir","ims2d1a","nics","clar_covers","properties","irregularities"], value='benzenoids', rows=1, layout={"width": "auto"})
                
        box = w.HBox([self.__element])
        display(box)
        
    
    def get_criterion (self) -> str:
        """ returns the JSON string corresponding to the criterion, an empty string if the criterion is not set """
        return str(self.__element.value)

### Definition of the display classes

In [7]:
class Table:
    """ This class allows for displaying a table containing benzenoid information """
    
    def __init__ (self):
        """ initializes the display tool """
        self.__data = {}
    
    
    def add_data (self, label, value) -> None:
        """ adds a label to display with its corresponding value """
        if label not in self.__data:
            self.__data[label] = []
        self.__data [label] += [value]
        
    
    def add_row (self, info: dict) -> None:
        """ adds a row based on the given information """           
        self.add_data ("Benzenoid id", info["idBenzenoid"])
        self.add_data ("InChI", info["inchi"])
        self.add_data ("Label", info["label"])
        self.add_data ("#hexagons", info["nbHexagons"])
        self.add_data ("#carbons", info["nbCarbons"])
        self.add_data ("#hydrogens", info["nbHydrogens"])
        self.add_data ("Weight", info["weight"])
        self.add_data ("Irregularity", info["irregularity"])
        
        
    def print (self):
        """ prints the table """
        self.__df = pd.DataFrame(self.__data)
        ip.display.display(self.__df)


In [8]:
class Table_Properties (Table):
    """ This class allows for displaying a table containing benzenoid information from properties query """

    def add_row (self, info: dict) -> None:
        """ adds a row based on the given information """
        super().add_row (info)
        yesno = ["no","yes"]
        self.add_data ("Catacondensed", yesno[info["catacondensed"]])
        self.add_data ("Coronoid", yesno[info["coronoid"]])
        self.add_data ("Coronenoid", yesno[info["coronenoid"]])
        self.add_data ("Symmetry", info["symmetry"])
        self.add_data ("#Kekulé structures", info["kekule"])
        self.add_data ("HOMO", info["homo"])
        self.add_data ("LUMO", info["lumo"])
        self.add_data ("Dipole moment", info["moment"])

In [9]:
class Table_Irregularities (Table):
    """ This class allows for displaying a table containing benzenoid information from irregularities query """
    
    def add_row (self, info: dict) -> None:
        """ adds a row based on the given information """
        super().add_row (info)
        self.add_data ("# solo", info["solo"])
        self.add_data ("# duo", info["duo"])
        self.add_data ("# trio", info["trio"])
        self.add_data ("# quartet", info["quartet"])
        

### Definition of the form class

In [10]:
def process (form):    
    demand_type = form.get_demand_type()

    if demand_type == "data":
        data = form.get_data()
        query = form.get_query()
        if query == "properties":
            d = Table_Properties()
        elif query == "irregularities":
            d = Table_Irregularities()
        else:
            d = Table()
        for molecule in data:
            d.add_row (molecule)
        d.print()

    elif demand_type == "count":
        print ("Number of molecules:",form.get_data())

    elif demand_type == "json":
        print ("JSON query:")
        print (form.get_json_string())
        
    elif demand_type == "result":
        print ("JSON result:")
        print (form.get_data())

In [15]:
class Form:
    
    def __init__ (self):
        """ initialise the form """
        self.__query = None         # the query
        self.__demand_type = None   # the type of demand
        self.__json_string = None   # the query as a JSON string
        self.__data = None          # the data related to the query (if the query succeeds)
        
        self.create_form()
        
        
    def create_form(self) -> None:
        """ creates the form allowing for choosing the values of desired criteria """
        self.__criteria = []

        # criteria about basic information
        self.__criteria.append (Int_Criterion(key="nbHexagons", description="# hexagons", min_value=1, max_value=9))
        self.__criteria.append (Int_Criterion(key="nbCarbons", description="# carbons", min_value=6, max_value=38))
        self.__criteria.append (Int_Criterion(key="nbHydrogens", description="# hydrogens", min_value=6, max_value=22))
        self.__criteria.append (Int_Criterion(key="idBenzenoid", description="Benzenoid id", min_value=0, max_value=8391))
        self.__criteria.append (Int_Criterion(key="solo", description="# solo", min_value=0, max_value=14))
        self.__criteria.append (Int_Criterion(key="duo", description="# duo", min_value=0, max_value=14))
        self.__criteria.append (Int_Criterion(key="trio", description="# trio", min_value=0, max_value=18))
        self.__criteria.append (Int_Criterion(key="quartet", description="# quartet", min_value=0, max_value=20))
        self.__criteria.append (Int_Criterion(key="kekule", description="# Kekulé structures", min_value=0, max_value=110))
        self.__criteria.append (Int_Criterion(key="catacondensed", description="Is catacondensed", min_value=0, max_value=1))
        self.__criteria.append (Int_Criterion(key="coronenoid", description="Is coronenoid", min_value=0, max_value=1))
        self.__criteria.append (Int_Criterion(key="coronoid", description="Is coronoid", min_value=0, max_value=1))
        self.__criteria.append (Float_Criterion(key="irregularity", description="Irregularity", min_value=0, max_value=1))
        self.__criteria.append (Float_Criterion(key="frequency", description="Frequency", min_value=0, max_value=3466))
        self.__criteria.append (Float_Criterion(key="intensity", description="Intensity", min_value=0, max_value=2742))
        self.__criteria.append (String_Criterion(key="inchi", description="InChI"))
        self.__criteria.append (String_Criterion(key="label", description="BenzAI label"))
        self.__criteria.append (String_Criterion(key="symmetry", description="Symmetry"))
        

        # query
        self.__criteria.append (Query_Criterion(key="query", description="query"))    

        # validation buttons
        count_btn = w.Button (description="Count")
        count_btn.on_click (self.perform_query)

        getdata_btn = w.Button (description="Get Data")
        getdata_btn.on_click (self.perform_query)

        getquery_btn = w.Button (description="Get JSON Query")
        getquery_btn.on_click (self.perform_query)
        
        getresult_btn = w.Button (description="Get JSON Result")
        getresult_btn.on_click (self.perform_query)
        
        box = w.HBox([count_btn, getdata_btn, getquery_btn, getresult_btn])
        display(box)
    
    
    def perform_query (self, btn) -> None:
        """ performs the query and sets the corresponding attributes """ 
        # we identify the type of demand
        if btn.description == "Count":
            self.__demand_type = "count"
        elif btn.description == "Get Data":
            self.__demand_type = "data"
        elif btn.description == "Get JSON Query":
            self.__demand_type = "json"
        elif btn.description == "Get JSON Result":
            self.__demand_type = "result"
        else:
            self.__demand_type = "unknown"

        # we build the JSON string
        self.__json_string = "{\n"
        for c in self.__criteria:
            if isinstance(c,Query_Criterion):
                self.__query = c.get_criterion()
            else:
                s = c.get_criterion()
                if len(s) > 0:
                    if len(self.__json_string) > 3:
                        self.__json_string += ",\n"
                    self.__json_string += "\t" + s
        self.__json_string += "\n}"

        if self.__demand_type in ["data", "result"]:
            response = requests.post("https://benzenoids.lis-lab.fr/find_"+self.__query, json= json.loads(self.__json_string))
        elif self.__demand_type == "count":
            response = requests.post("https://benzenoids.lis-lab.fr/count_"+self.__query, json= json.loads(self.__json_string))
        
        if self.__demand_type in ["data", "count", "result"]:
            if response.status_code == 200:
                self.__data = response.json()
                
        process(self)

        
    def get_data (self):
        """ returns the data related to the query """
        return self.__data
    
    
    def get_query (self):
        """ returns the desired query """
        return self.__query

    
    def get_json_string (self):
        """ returns the JSON string related to the desired query """
        return self.__json_string
    

    def get_demand_type (self):
        """ returns the type of the current demand """
        return self.__demand_type

## Filling the form

In [16]:
form = Form()


JSON result:
[{'idBenzenoid': 3532, 'inchi': '1S/C14H10/c1-3-7-13-11(5-1)9-10-12-6-2-4-8-14(12)13/h1-10H', 'label': '1-3-4', 'nbHexagons': 3, 'nbCarbons': 14, 'nbHydrogens': 10, 'irregularity': 0.8, 'graphFile': 'p DIMACS 14 16 3\ne 0_0 1_1\ne 0_0 -1_1\ne 1_1 1_2\ne 1_2 0_3\ne 0_3 -1_2\ne 0_3 0_4\ne -1_2 -1_1\ne -1_2 -2_3\ne -3_2 -2_3\ne -3_2 -4_3\ne -2_3 -2_4\ne -2_4 -3_5\ne -2_4 -1_5\ne -3_5 -4_4\ne -4_4 -4_3\ne 0_4 -1_5\nh 0_0 1_1 1_2 0_3 -1_2 -1_1 \nh -3_2 -2_3 -2_4 -3_5 -4_4 -4_3 \nh -1_2 0_3 0_4 -1_5 -2_4 -2_3 \n', 'geometry': 'C 0.00000 -0.68162 2.09919\nC 0.00000 -1.42610 0.86923\nC 0.00000 0.68162 2.09919\nC 0.00000 -0.73006 -0.38141\nC 0.00000 -2.84366 0.88143\nC 0.00000 0.73006 -0.38141\nC 0.00000 -1.50189 -1.57087\nC 0.00000 1.42610 0.86923\nC 0.00000 1.50189 -1.57087\nC 0.00000 2.84366 0.88143\nC 0.00000 -3.56876 -0.29788\nC 0.00000 -2.88823 -1.53355\nC 0.00000 2.88823 -1.53355\nC 0.00000 3.56876 -0.29788\nH 0.00000 -1.23461 3.03409\nH 0.00000 1.23461 3.03409\nH 0.00000 -3